
## Mini PJT.

- `titanic` competition을 도전해봅시다!

- 이번 프로젝트에서는 간단한 분류 문제를 풀어봅니다.

- sklearn으로 머신러닝 모델을 구현해봅니다.

- Machine Learning Workflow를 따라가봅니다.


Source : https://www.kaggle.com/c/titanic

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# titanic data 불러오기


### Data Preprocessing

1. 결측치 처리


2. feature selection (분석에 사용하지 않을 column 제거)

In [ ]:
# titanic data에서 missing value를 찾아봅니다.


In [ ]:
# Embarked column이 NaN인 row를 찾습니다.


In [ ]:
# missing value를 handling 합니다.
# column을 지울까요 / 채울까요?


### Feature Engineering

1. Categorical feature encoding

2. Normalization

In [ ]:
# categorical feature --> One-hot Encoding


In [ ]:
# Normalization --> Min-Max scaling


### Training 

In [ ]:
# sklearn에서 배웠던 분류 모델들을 불러와봅니다.


### Test (Predict)

In [ ]:
test

In [ ]:
# test data에 같은 feature engineering을 적용해줍니다.


In [ ]:
# 결과 파일인 submission.csv를 생성합니다.
submission = 

- 모든 학습이 끝나면 결과를 가지고 제출해볼 수 있습니다.

- 만든 모델 중에 가장 test 성능이 좋은 하나를 제출해볼까요?

[제출하러가기] https://www.kaggle.com/c/titanic